# Information Extraction - Assignment
This assignment is based on the Information Extraction lecture and the lab.


In [1]:
import os
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from statistics import mode
import itertools
import json
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.tag import StanfordNERTagger
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
from difflib import SequenceMatcher

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [2]:
from google.colab import files
uploaded = files.upload()

Saving football_players.txt to football_players.txt


In [3]:

inputfile='football_players.txt' #Location of the file
buf=open(inputfile, encoding="UTF-8")
list_of_doc=buf.read().split('\n')
# Removing all empty lines i.e empty lines between paragraph and converting every value to String
l = []
for i in list_of_doc:
    if len(i) != 0:
        l.append(i)
list_of_doc = l

print(list_of_doc)

["Cristiano Ronaldo dos Santos Aveiro, ComM, GOIH (born 5 February 1985) is a Portuguese professional footballer who plays for Spanish club Real Madrid and the Portugal national team. He is a forward and serves as captain for Portugal. In 2008, he won his first Ballon d'Or and FIFA World Player of the Year awards. He then won the FIFA Ballon d'Or in 2013 and 2014. In 2015, Ronaldo scored his 500th senior career goal for club and country. Often ranked as the best player in the world, Ronaldo was named the best Portuguese player of all time by the Portuguese Football Federation, during its 100th anniversary celebrations in 2015. He is the only player to win four European Golden Shoe awards. One of the most marketable athletes in sport, in 2016 Forbes named Ronaldo the world's best paid athlete. In June 2016, ESPN ranked him the world's most famous athlete. Ronaldo began his club career playing for Sporting CP, before signing with Manchester United at age 18 in 2003. After winning his fir

# Task 1 (10 Marks)
Write a function that takes each document and performs:
1) sentence segmentation 2) tokenization 3) part-of-speech tagging

Please keep in mind that the expected output is a list within a list as shown below.


In [0]:
def ie_preprocess(document):
    #This function pos tag the sentences and stores in the list'''
    # Convert string into list of sentences
    sentences = nltk.sent_tokenize(document)
    pos_sentences = []
    try:
      sentences = [nltk.word_tokenize(sent) for sent in sentences]
      pos_sentences = [nltk.pos_tag(sent) for sent in sentences]
      return pos_sentences
    except:
      print("The input to the function must be of string Type")



Run the following code to check your result for the first document (Ronaldo).

In [5]:
first_doc=list_of_doc[0]
# Calling the ie_preprocess function
pos_sent=ie_preprocess(first_doc)
pos_sent[1]


[('He', 'PRP'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('forward', 'NN'),
 ('and', 'CC'),
 ('serves', 'NNS'),
 ('as', 'IN'),
 ('captain', 'NN'),
 ('for', 'IN'),
 ('Portugal', 'NNP'),
 ('.', '.')]

Expected output
 [...[('He', 'PRP'),
  ('is', 'VBZ'),
  ('a', 'DT'),
  ('forward', 'NN'),
  ('and', 'CC'),
  ('serves', 'NNS'),
  ('as', 'IN'),
  ('captain', 'NN'),
  ('for', 'IN'),
  ('Portugal', 'NNP'),
  ('.', '.')], ...]

# Task 2 (20 Marks)
Write a function that will take the list of tokens with POS tags for each sentence and returns the named entities (NE). 

Hint: Use binary=True while calling NE chunk function

In [6]:
import nltk
def named_entity_finding(p_sent , x=True, y="NE"):
    #Code goes here
    # This is check if the return needs all NE or specific NE - default type is PERSON.
    if y != "NE":
        x=False
        if y=="NE":
            y = "PERSON"
    tree = nltk.ne_chunk(p_sent, binary=x)
    #print(tree)
    named_entities = []
    for subtree in tree.subtrees():
        #print(subtree.label())
        if (subtree.label() == "NE") :
          entity = ""
          for leaf in subtree.leaves():
              entity = entity + leaf[0] + " "
          named_entities.append(entity.strip())
    return (named_entities)

#named_entity_finding()
pos_sents=ie_preprocess(list_of_doc[0])
print(named_entity_finding(pos_sents[0]))

['Cristiano Ronaldo', 'Santos Aveiro', 'ComM', 'GOIH', 'Portuguese', 'Spanish', 'Real Madrid', 'Portugal']


Expected output ['Cristiano Ronaldo',
 'Santos Aveiro',
 'ComM',
 'GOIH',
 'Portuguese',
 'Portuguese',
 'Spanish',
 'Real Madrid',
 'Portugal']

# Task 3 (10 Marks)

Now use the named_entity_finding() function to extract all NEs for each document.

Hint: pos_sents holds the list of lists of tokens with POS tags

In [7]:
def NE_flat_list_fn(document):
    """This function is extracts named entities in entire document and returns all unique NE as flattened list"""
    NE=[]
    try:
        # Loop through each docudment 1-10
        for i in document:
            # POS tag the document
            pos_sents = ie_preprocess(i)
            #print(pos_sents)
            for pos_sent in pos_sents:
                # For each tagged sentence in document find named entity
                entity = named_entity_finding(pos_sent)
                #print(entity)
                if len(entity) != 0:
                    NE.append(entity)

        
        # This function flattens the list
        NE_flat_list = list(itertools.chain.from_iterable(NE))
        return NE_flat_list
    except:     
        print("Please make sure that input to the function is off type string")

        
ne_flat = NE_flat_list_fn(list_of_doc)
ne_flat = list(set(ne_flat))
print(ne_flat)

['Portuguese Football Federation', 'Tottenham Hotspur', 'Preston North End', 'Netherlands', 'Rivaldo', 'Portugal', 'Europe', 'Golden Ball Award', 'UNICEF UK', 'Cristiano Ronaldo', 'Brazil', 'FIFA Confederations Cup', 'Ballon', 'England', 'PSG', 'Neymar Jr.', 'OBE', 'UEFA European', 'South American Youth Championship', 'Pelé', 'David Robert Joseph Beckham', 'Steven Gerrard', 'Michel Platini', 'Santos', 'Ajax', 'Swedish', 'Beckham', 'Wayne Rooney', 'FIFPro World', 'BBC', 'Flamengo', 'Mesut Özil', 'Born', 'Golden Ball', 'European Golden Shoe', 'Ronaldinho', 'Germany', 'Golden Shoe', 'Lionel Andrés', 'FIFA Puskás Award', 'Match', 'Zinedine Zidane', 'PFA Players', 'UEFA Best Player', 'FC Barcelona', 'Dagens Nyheter', 'Real Madrid', 'Southampton', 'David Trezeguet', 'Spain', 'Paris', 'UEFA Euro', 'Malmö FF', 'Italy', 'South American Footballer', 'Harry Redknapp', 'IFFHS World', 'LA Galaxy', 'United States', 'Wales', 'FIFA World Youth Championship', 'Andrés Iniesta Luján', 'FWA Footballer', '

# Task 4 (40 Marks)

Write functions to extract the name of the player, country of origin and date of birth as well as the following relations: team(s) of the player and position(s) of the player.

Hint: Use the re.compile() function to create the extraction patterns

Reference: https://docs.python.org/3/howto/regex.html

In [8]:
def name_of_the_player(doc):
    #code goes here
    # Hint: Use the named_entity_finding() function
    t = []
    name = []
    
    try:
        pos_sents = ie_preprocess(doc)
        #print(pos_sents[0])
        tree = nltk.ne_chunk(pos_sents[0])
        #print(tree)
        for subtree in tree.subtrees():
            if subtree.label() == "PERSON":
                entity = ""
                for leaf in subtree.leaves():
                    entity = entity + leaf[0] + " "
                name.append(entity.strip())

        return " ".join(name)
         
    except:
       print("The document has to be of type string")
try:
   print(name_of_the_player(list_of_doc[1]))
except:
   print("Please make sure give list index is right")


Lionel Andrés


Here we can extract player name using standfor NE tagger as well. Standford works slightly better than NLTK default tagger. Moreover, Standfor NE tagger detects the surname as well. E.g: NLTK default - Cristiano Santos Aveiro : retuns Cristiano as PERSON and Ronaldo as ORGANIZATION. Hence using standford package will help in this."""
        

In [9]:
def country_of_origin(doc):
    
    """ Three levels of filter is applied here. Because NLTK NE tagger dosent recognise all the country names. Moreover, 
    we can use look up to get the country name (because the text contains like spanish that can be converted to Spain).
    LEVEL 1 - Used NLTK tagger to find Named Entity. If not found go to LEVEL - 2
    LEVEL 2 - Used Regex to check for word with profesional footballer. If not found go to LEVEL - 3
    LEVEL 3 - USed Regex to match word with National Team.
    Note - Not used Look up here. This type of Levels is useful in a large documents"""
    
    sentence = sent_tokenize(doc)[0]
    tm = []
    seq = []
    # LEVEL 2 (look for prrofesssional footballer.)
    # This uses regex to find the country of origin if NLTK fails to detect GPE or LOCATION tag.
    m = re.search(r'((?:\w+\W+){,3})(footballer)', sentence)
    s = "".join(list(m.groups()))
    mal = re.search(r'((\w+ ){1})professional', s)
    # Take the first group most probably the country name
    try:
      mal = mal.groups()[0]
    except:
      mal = None

    # LEVEL - 1 (Primary lever, get NE for 1st sentence and look for GPE)
    # Get tag for first sentence
    tag = ie_preprocess(s)[0]
    tree = nltk.ne_chunk(tag)
    born = []
    # Check for GPE tag if any (sometimes NLTK dosent detect GPE).
    for subtree in tree.subtrees():
      if subtree.label() == 'GPE':
        entity = ""
        for leaf in subtree.leaves():
          entity = entity + leaf[0] + " "
        born.append(entity.strip())
        
    # LEVEL 3 (if nowhere found then check for words with national team.)
    match = re.compile(r'((?:[\S,]+\s+){0,1})national team')
    for i, sent in enumerate(sent_tokenize(doc)):
      if i == 2:
        break
      team = match.findall(sent)
      if len(team) != 0:
        tm.append(team[0])
    # If NLTK fails to detect then use Regex re.search(r'((?:\w+\W+){,3})(footballer)', sentence) - i.e take 3 words before 
    # the word footballer and take the first group.
    if len(born) == 0:
      if len(mal) != 0:
        born = nltk.word_tokenize(mal) 
      elif len(tm) != 0:
        born = tm
        
        
    # Lookup of all the countries - can use all country name as look up but included only few.
    # sequence matcher is used to find the country in look up.
    l = ['Spain', 'England', 'Portugal', 'Wales', 'England', 'Brazil', 'Germany', 'Argentina', 'Sweden']
    for x in l:
      seq.append(SequenceMatcher(None, born[0], x).ratio())
    final = l[seq.index(max(seq))]
    return final
  
print(country_of_origin(list_of_doc[5]))
                

Sweden


In [10]:
def date_of_birth(doc):
    
    """Used regex to find the date of bith. First extract the sentence containing born and find the matching pattern."""
    
    sentence = sent_tokenize(doc)[0]
    match = re.compile(r'born\b\s*((?:\S+\s+){0,3})')
    born = match.findall(sentence)[0]
    born = re.sub('\W+',' ', born )
    
    return born


try:
    print(date_of_birth(list_of_doc[1]))
except:
    print("Please make sure the list indexing is right")

24 June 1987 


In [16]:
def team_of_the_player(doc):
    
    """This functions returns teams of the player. Note Some of the players dosent have national team information
    (for sentence 4). Hence, assumed as player plays for his country of origin (This may not be true for always). """
    
    sentence = sent_tokenize(doc)
    tm = []
    club = []
    club2 = []
    
    try:
        tag = ie_preprocess(doc)
        # Check for only first 2 sentences. Saves time and efficent.
        for i, sent in enumerate(sentence):
            if i == 2:
                break
            # Extract the national team of the player using below regex.
            match = re.compile(r'((?:[\S,]+\s+){0,1})national team')
            
            # This is to extract the clubs of the player.
            match_club = re.compile(r'club\s+((?:[\S,]+\s*){0,2})')
            
            
            # Find the national team
            team = match.findall(sent)
            
            # If club keyword found in first couple of sentences. Extract those clubs.
            if len(match_club.findall(sent)) != 0:
                for f in match_club.findall(sent):
                    club2.append(f)
            
            # Search of clubs. This can be found by checking for organization by calling Named Entity function and type is 
            # ORGANIZATION.
            club.append(named_entity_finding(tag[i], False, "ORGANIZATION"))
            if len(team) != 0:
                l = team[0]+"national team"
                tm.append(l)
            
        # Find unique names because sentence may have same names multiple times.
        if len(list(set(tm))) != 0:
            national_team = list(set(tm))[0]
        else:
            national_team = country_of_origin(doc) +" national team"

        # Flattens the list
        club = list(itertools.chain.from_iterable(club))
        
        for i, s in enumerate(club2):
            club2[i] = club2[i].rstrip()
        if len(list(set(club).intersection(club2))) != 0:
            club = list(set(club).intersection(club2))
        club.append(national_team)

        club = list(set(club).difference(nltk.word_tokenize(sentence[0])[0:6]))
        club = club+club2
        
        return club
    except:
        print("Please make sure the data given to this function is of type string")


try:
    print(team_of_the_player(list_of_doc[0]))
except:
    print("Please make sure the given list index is right.")

['Portugal national team', 'Real Madrid']


In [17]:
def position_of_the_player(doc):
    
    """ This function extracts position of the player. Can return multiple values if the player played in 
    different clubs or teams as different position. Hence the return type is list."""


    # This is look up of all postions available in football
    pos = ["forward", "captian", "attacking midfielder", "striker", "winger", "central midfielder", "defensive tackle", "defensive end"]
    player_position = []
    try:
        sent = sent_tokenize(doc)
        for i, sent in enumerate(sent):
            for x in pos:
                # Find matching in look up and sentence.
                regex = re.compile(r'\b({0})\b'.format(x), flags=re.IGNORECASE)
                
                # if result is true append to a list
                r = bool(regex.search(sent))
                
                if r == True:
                    player_position.append(x)
        
        return list(set(player_position))
    except:
        print("Please make sure that input to the function is string")

try:
    print(position_of_the_player(list_of_doc[3]))
except:
    print("Maximum length of document is 10 i.e. index 9")

['forward', 'attacking midfielder']


Execute the below command to check your fuction


In [36]:
date_of_birth(list_of_doc[2])

'5 February 1992 '

Expected output '5 February 1992'

# Task 5 (10 Marks)

Write a function using the outputs from the previous functions to generate JSON-LD output as follows.

Reference: https://json-ld.org/primer/latest/

{ "@id": "http://my-soccer-ontology.com/footballer/name_of_the_player",

    "name": "",
    "born": "",
    "country": "",
    "position": [
        { "@id": "http://my-soccer-ontology.com/position",
            "type": ""
        }
     ]   
     "team": [
        { "@id": "http://my-soccer-ontology.com/team",
            "name": ""
        }   
     ]
}


In [0]:
def data_generator(doc):
    
    data = [name_of_the_player(doc), date_of_birth(doc), country_of_origin(doc), position_of_the_player(doc), team_of_the_player(doc)]
    
    return data


data = data_generator(list_of_doc[0])

In [20]:
def generate_jsonld(arg, con=True):
    if con == True:

        ld = { "@id": "http://my-soccer-ontology.com/footballer/"+arg[0],

            "name": arg[0],
            "born": arg[1],
            "country": arg[2],
            "position": [
                { "@id": "http://my-soccer-ontology.com/position/",
                    "type": arg[3]
                }
             ],   
             "team": [
                { "@id": "http://my-soccer-ontology.com/team/",
                    "name": arg[4]
                }   
             ]
        }

        return json.dumps(ld)
    
    elif con == False:
        
        ld = { "@id": "http://my-soccer-ontology.com/footballer/"+arg[0],

            "name": arg[0],
            "born": arg[1],
            "country": arg[2],
            "position": [
                { "@id": "http://my-soccer-ontology.com/position",
                    "type": arg[3]
                }
             ],   
             "team": [
                { "@id": "http://my-soccer-ontology.com/team",
                    "name": arg[4]
                },
             ],
            "Debut Year": arg[5][0],
            "Debut Age": arg[5][1]
        }

        return json.dumps(ld)
        



data = data_generator(list_of_doc[0])
print(generate_jsonld(data))

#print("Please make sure that list index given is right")


{"@id": "http://my-soccer-ontology.com/footballer/Cristiano Santos Aveiro", "name": "Cristiano Santos Aveiro", "born": "5 February 1985 ", "country": "Portugal", "position": [{"@id": "http://my-soccer-ontology.com/position/", "type": ["forward"]}], "team": [{"@id": "http://my-soccer-ontology.com/team/", "name": ["Portugal national team", "Real Madrid"]}]}


# Task 6 (10 Marks)
Identify one other relation (besides team and player) and write a function to extract this. Also extend the JSON-LD output accordingly.

Here I have taken debut year and debut age of the player. Example - Neymar da Silva Santos Júnior debut at the age of 17 with a club 'Santos'. But this data is not available for all the players. Hence, if debut year or debut age is not present I have added "NA".

In [21]:
def relation_debutYearAge(doc):
    
    """This function extracts the debut year and debut age of the player. 
    Note - this information if not available for all the players"""
    
    sent = sent_tokenize(doc)
    deb = []
    ag = []
    # For each sentence check if the debut key is present or not
    for se in sent:
        sp_sent = se.split()
        
        if "debut" in sp_sent:
            # If debut year is present jecy for 4 digit number.
            date = re.findall('\d{4}', " ".join(sp_sent))
            if len(date) !=0:
                deb.append(date[0])
        
        # Check for debut and age key in a sentence - 
        if "debut" in sp_sent and "aged" in sp_sent:
            # Search for 2 digit number and append to the list.
            age = re.findall('\d{2}', " ".join(sp_sent))
            ag.append(age[0])       
    
    # If the above process dosent return anything, insert year and age is not available.
    if len(deb) == 0:
        deb.append("Not Available")
    if len(ag) == 0:
        ag.append("Not Available")

    return [deb[0], ag[0]]

relation_debutYearAge(list_of_doc[1])

def data_generator(doc):
    
    data = [name_of_the_player(doc), date_of_birth(doc), country_of_origin(doc), position_of_the_player(doc), team_of_the_player(doc), relation_debutYearAge(doc)]
    return data
  
  
data = data_generator(list_of_doc[3])
print(generate_jsonld(data, False))


{"@id": "http://my-soccer-ontology.com/footballer/Ronaldo Assis Moreira", "name": "Ronaldo Assis Moreira", "born": "21 March 1980 ", "country": "Brazil", "position": [{"@id": "http://my-soccer-ontology.com/position", "type": ["forward", "attacking midfielder"]}], "team": [{"@id": "http://my-soccer-ontology.com/team", "name": ["Brazil national team", "ambassador for", "FC Barcelona"]}], "Debut Year": "Not Available", "Debut Age": "Not Available"}
